In [ ]:
"""
Script AMÉLIORÉ pour générer des morphings sur la base de données LFW
✨ Avec suivi visuel en temps réel et rendus artistiques
"""

import numpy as np
import cv2
import dlib
import matplotlib.pyplot as plt
import os
from pathlib import Path
import urllib.request
import bz2
from sklearn.datasets import fetch_lfw_people
from itertools import combinations
from tqdm import tqdm
import time
from datetime import datetime, timedelta
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import imageio
from matplotlib.patches import Rectangle, FancyBboxPatch
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# ==================== CLASSE DE SUIVI VISUEL ====================
class MorphingTracker:
    """Gestionnaire de suivi et visualisation en temps réel"""
    
    def __init__(self, total_morphs):
        self.total = total_morphs
        self.current = 0
        self.success = 0
        self.failures = 0
        self.start_time = time.time()
        self.recent_times = []
        
        # Préparer la figure de suivi
        plt.ion()  # Mode interactif
        self.fig = plt.figure(figsize=(16, 10))
        self.gs = GridSpec(3, 3, figure=self.fig)
        
        # Axes pour différentes visualisations
        self.ax_preview = self.fig.add_subplot(self.gs[0:2, 0:2])  # Grande prévisualisation
        self.ax_stats = self.fig.add_subplot(self.gs[0, 2])  # Statistiques
        self.ax_speed = self.fig.add_subplot(self.gs[1, 2])  # Graphique de vitesse
        self.ax_gallery = [
            self.fig.add_subplot(self.gs[2, 0]),
            self.fig.add_subplot(self.gs[2, 1]),
            self.fig.add_subplot(self.gs[2, 2])
        ]  # Derniers morphings
        
        self.recent_morphs = []
        self.speed_history = []
        
        self.fig.suptitle('🎭 GÉNÉRATION DE MORPHINGS - SUIVI EN TEMPS RÉEL', 
                         fontsize=16, fontweight='bold')
        plt.tight_layout()
        
    def update(self, morph_img, person_a, person_b, alpha, duration, success=True):
        """Met à jour l'affichage avec le dernier morphing"""
        self.current += 1
        if success:
            self.success += 1
        else:
            self.failures += 1
        
        self.recent_times.append(duration)
        if len(self.recent_times) > 20:
            self.recent_times.pop(0)
        
        # Calculs de performance
        elapsed = time.time() - self.start_time
        avg_time = np.mean(self.recent_times) if self.recent_times else 1
        remaining = (self.total - self.current) * avg_time
        speed = self.current / elapsed if elapsed > 0 else 0
        
        # Mise à jour de la prévisualisation principale
        self.ax_preview.clear()
        if morph_img is not None and success:
            self.ax_preview.imshow(cv2.cvtColor(morph_img, cv2.COLOR_BGR2RGB))
            self.ax_preview.set_title(
                f'🎨 {person_a[:15]} ➜ {person_b[:15]}\n'
                f'Alpha: {alpha:.2f} | Temps: {duration:.2f}s',
                fontsize=12, fontweight='bold', color='darkblue'
            )
        self.ax_preview.axis('off')
        
        # Mise à jour des statistiques
        self.ax_stats.clear()
        self.ax_stats.axis('off')
        
        stats_text = f"""
╔════════════════════════════╗
║      STATISTIQUES          ║
╠════════════════════════════╣
║ Progression: {self.current}/{self.total}
║ ✅ Succès: {self.success}
║ ❌ Échecs: {self.failures}
║ 
║ ⏱️  Temps écoulé: {timedelta(seconds=int(elapsed))}
║ ⏳ Temps restant: {timedelta(seconds=int(remaining))}
║ 
║ ⚡ Vitesse: {speed:.2f} img/s
║ 📊 Moyenne: {avg_time:.2f}s/img
║ 
║ 📈 Taux de succès: {100*self.success/max(1,self.current):.1f}%
╚════════════════════════════╝
        """
        
        self.ax_stats.text(0.05, 0.95, stats_text, 
                          transform=self.ax_stats.transAxes,
                          fontsize=9, verticalalignment='top',
                          fontfamily='monospace',
                          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
        
        # Graphique de vitesse
        self.speed_history.append(speed)
        if len(self.speed_history) > 50:
            self.speed_history.pop(0)
        
        self.ax_speed.clear()
        if len(self.speed_history) > 1:
            self.ax_speed.plot(self.speed_history, color='royalblue', linewidth=2)
            self.ax_speed.fill_between(range(len(self.speed_history)), 
                                       self.speed_history, alpha=0.3)
            self.ax_speed.set_title('⚡ Vitesse de génération', fontsize=10, fontweight='bold')
            self.ax_speed.set_ylabel('Images/s')
            self.ax_speed.set_xlabel('Temps')
            self.ax_speed.grid(True, alpha=0.3)
        
        # Galerie des derniers morphings
        if morph_img is not None and success:
            self.recent_morphs.append(morph_img)
            if len(self.recent_morphs) > 3:
                self.recent_morphs.pop(0)
            
            for idx, ax in enumerate(self.ax_gallery):
                ax.clear()
                if idx < len(self.recent_morphs):
                    ax.imshow(cv2.cvtColor(self.recent_morphs[idx], cv2.COLOR_BGR2RGB))
                    ax.set_title(f'Morphing #{self.success - len(self.recent_morphs) + idx + 1}', 
                               fontsize=8)
                ax.axis('off')
        
        # Rafraîchir l'affichage
        plt.pause(0.01)
    
    def close(self):
        """Ferme la fenêtre de suivi"""
        plt.ioff()
        plt.close(self.fig)

# ==================== CONFIGURATION ====================
LOCAL_DATA_DIR = Path("./dlib_models")
LOCAL_DATA_DIR.mkdir(exist_ok=True)
PREDICTOR_PATH = LOCAL_DATA_DIR / "shape_predictor_68_face_landmarks.dat"

# Configuration du dataset
MIN_FACES_PER_PERSON = 30
RESIZE_FACTOR = 0.5
SIZE = 128

# Configuration de génération
OUTPUT_DIR = Path("./morphing_results")
OUTPUT_DIR.mkdir(exist_ok=True)
GIFS_DIR = OUTPUT_DIR / "gifs"
GIFS_DIR.mkdir(exist_ok=True)
GRIDS_DIR = OUTPUT_DIR / "grids"
GRIDS_DIR.mkdir(exist_ok=True)

# Options de génération
MODE = "sample"  # Changé à "sample" par défaut pour des tests rapides
NUM_SAMPLES = 20
ALPHA_VALUES = [0.5]
SAVE_INDIVIDUAL = True
CREATE_GRID = True
CREATE_GIFS = True  # 🎬 Nouveau: créer des GIFs animés
CREATE_SMOOTH_TRANSITIONS = True  # 🎨 Nouveau: transitions fluides
GRID_SIZE = (4, 5)

# Bannière de démarrage stylisée
print("""
╔═══════════════════════════════════════════════════════════════════════╗
║                                                                       ║
║            🎭  MORPHING STUDIO - VERSION AMÉLIORÉE  🎨               ║
║                                                                       ║
║              Génération artistique de morphings faciaux              ║
║                      avec suivi en temps réel                        ║
║                                                                       ║
╚═══════════════════════════════════════════════════════════════════════╝
""")

print(f"""
⚙️  CONFIGURATION DU PROJET:
╭──────────────────────────────────────────────────────────╮
│ 📁 Mode de génération    : {MODE:30s} │
│ 🎯 Nombre d'échantillons : {NUM_SAMPLES:<30d} │
│ 🎨 Valeurs Alpha         : {str(ALPHA_VALUES):30s} │
│ 📐 Taille des images     : {SIZE}x{SIZE:25s} │
│ 💾 Dossier de sortie     : {str(OUTPUT_DIR):30s} │
│ 🎬 Créer des GIFs        : {'Oui' if CREATE_GIFS else 'Non':30s} │
│ 🖼️  Créer grille          : {'Oui' if CREATE_GRID else 'Non':30s} │
│ ✨ Transitions fluides   : {'Oui' if CREATE_SMOOTH_TRANSITIONS else 'Non':30s} │
╰──────────────────────────────────────────────────────────╯
""")

In [ ]:
# ==================== TÉLÉCHARGEMENT ET CHARGEMENT ====================

def download_dlib_predictor():
    """Télécharge le fichier shape_predictor_68_face_landmarks.dat si nécessaire"""
    if PREDICTOR_PATH.exists():
        print("✓ Modèle Dlib déjà présent")
        return
    
    print("\n📥 Téléchargement du modèle Dlib...")
    url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
    compressed_file = LOCAL_DATA_DIR / "shape_predictor_68_face_landmarks.dat.bz2"
    
    try:
        # Barre de progression pour le téléchargement
        def show_progress(block_num, block_size, total_size):
            downloaded = block_num * block_size
            percent = min(100, downloaded * 100 / total_size)
            bar_length = 40
            filled = int(bar_length * percent / 100)
            bar = '█' * filled + '░' * (bar_length - filled)
            print(f'\r   [{bar}] {percent:.1f}%', end='', flush=True)
        
        urllib.request.urlretrieve(url, compressed_file, show_progress)
        print("\n   Décompression...")
        
        with bz2.BZ2File(compressed_file, 'rb') as f_in:
            with open(PREDICTOR_PATH, 'wb') as f_out:
                f_out.write(f_in.read())
        
        compressed_file.unlink()
        print("   ✓ Modèle Dlib prêt !\n")
        
    except Exception as e:
        raise RuntimeError(f"❌ Erreur lors du téléchargement : {e}")

def load_lfw_compact():
    """Charge le dataset LFW avec animation"""
    print("\n📥 Chargement du dataset LFW...")
    
    try:
        lfw_people = fetch_lfw_people(
            min_faces_per_person=MIN_FACES_PER_PERSON,
            resize=RESIZE_FACTOR,
            color=False
        )
        
        images = lfw_people.images
        labels = lfw_people.target
        target_names = lfw_people.target_names
        
        n_samples, h, w = images.shape
        n_classes = len(target_names)
        
        print(f"""
✅ Dataset chargé avec succès !
╭──────────────────────────────────────────╮
│ 📊 Total images      : {n_samples:6d}        │
│ 👥 Nombre de personnes: {n_classes:6d}        │
│ 📐 Dimensions        : {h}x{w:14s} │
╰──────────────────────────────────────────╯
""")
        
        return images, labels, target_names
        
    except Exception as e:
        raise RuntimeError(f"❌ Erreur lors du chargement: {e}")

# Initialisation
download_dlib_predictor()

try:
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(str(PREDICTOR_PATH))
    print("✓ Modèle Dlib chargé et prêt\n")
except Exception as e:
    raise RuntimeError(f"❌ Impossible de charger Dlib: {e}")

images, labels, target_names = load_lfw_compact()

In [ ]:
# ==================== FONCTIONS DE MORPHING DE BASE ====================

def get_landmarks(img_gray, detector, predictor, upsample_times=0):
    """Détecte les landmarks faciaux"""
    dets = detector(img_gray, upsample_times)
    if len(dets) == 0:
        return None
    shape = predictor(img_gray, dets[0])
    pts = np.zeros((68, 2), dtype=np.int32)
    for i in range(68):
        pts[i] = (shape.part(i).x, shape.part(i).y)
    return pts

def add_corner_points(points, w, h):
    corners = np.array([
        [0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1],
        [w // 2, 0], [w - 1, h // 2], [w // 2, h - 1], [0, h // 2]
    ], dtype=np.int32)
    return np.concatenate([points, corners], axis=0)

def clamp_points(points, w, h):
    pts = np.array(points, dtype=np.float32)
    pts[:, 0] = np.clip(pts[:, 0], 0, w - 1)
    pts[:, 1] = np.clip(pts[:, 1], 0, h - 1)
    return pts

def find_point_index(points, pt, tol=3.0):
    pts = np.asarray(points, dtype=np.float32)
    dists = np.linalg.norm(pts - np.asarray(pt, dtype=np.float32), axis=1)
    idx = int(np.argmin(dists))
    if dists[idx] <= tol:
        return idx
    return None

def triangle_completely_inside(t, w, h):
    for (x, y) in t:
        if x < 0 or x >= w or y < 0 or y >= h:
            return False
    return True

def apply_affine_transform(src, src_tri, dst_tri, size):
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    dst = cv2.warpAffine(src, warp_mat, (int(size[0]), int(size[1])),
                         None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    return dst

def morph_triangle(img1, img2, img_morphed, t1, t2, t_morphed, alpha):
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t_morphed]))

    if r1[2] <= 0 or r1[3] <= 0 or r2[2] <= 0 or r2[3] <= 0 or r[2] <= 0 or r[3] <= 0:
        return

    t1_rect = [(t1[i][0] - r1[0], t1[i][1] - r1[1]) for i in range(3)]
    t2_rect = [(t2[i][0] - r2[0], t2[i][1] - r2[1]) for i in range(3)]
    t_rect = [(t_morphed[i][0] - r[0], t_morphed[i][1] - r[1]) for i in range(3)]

    img1_rect = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
    img2_rect = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]]

    if img1_rect.size == 0 or img2_rect.size == 0:
        return

    size_rect = (r[2], r[3])

    warp_img1 = apply_affine_transform(img1_rect, t1_rect, t_rect, size_rect)
    warp_img2 = apply_affine_transform(img2_rect, t2_rect, t_rect, size_rect)

    img_rect = (1.0 - alpha) * warp_img1 + alpha * warp_img2

    mask = np.zeros((r[3], r[2]), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(t_rect), 1.0, 16, 0)

    y, x, w_rect, h_rect = r[1], r[0], r[2], r[3]
    img_morphed[y:y+h_rect, x:x+w_rect] = img_morphed[y:y+h_rect, x:x+w_rect] * (1 - mask[:, :, None]) + img_rect * mask[:, :, None]

def prepare_points_for_image(img_gray, detector, predictor, w, h):
    pts = get_landmarks(img_gray, detector, predictor, upsample_times=0)
    if pts is None:
        grid_x = np.tile(np.linspace(w*0.25, w*0.75, 17), (4,))
        grid_y = np.repeat(np.linspace(h*0.25, h*0.75, 4), 17)
        grid = np.vstack([grid_x[:68], grid_y[:68]]).T.astype(np.int32)
        pts = grid
    pts = clamp_points(pts, w, h)
    pts = add_corner_points(pts.astype(np.int32), w, h)
    return pts.astype(np.float32)

def morph_faces(imgA, imgB, alpha=0.5):
    """Morphe deux visages"""
    # Redimensionner
    imgA_resized = cv2.resize(imgA, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    imgB_resized = cv2.resize(imgB, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    
    # Convertir en uint8
    if imgA_resized.dtype != np.uint8:
        imgA_resized = (imgA_resized * 255).astype(np.uint8)
    if imgB_resized.dtype != np.uint8:
        imgB_resized = (imgB_resized * 255).astype(np.uint8)
    
    # Convertir en couleur
    imgA_color = cv2.cvtColor(imgA_resized, cv2.COLOR_GRAY2BGR).astype(np.float32)
    imgB_color = cv2.cvtColor(imgB_resized, cv2.COLOR_GRAY2BGR).astype(np.float32)

    # Préparer les points
    ptsA = prepare_points_for_image(imgA_resized, detector, predictor, SIZE, SIZE)
    ptsB = prepare_points_for_image(imgB_resized, detector, predictor, SIZE, SIZE)

    # Points morphés
    points_morphed = (1.0 - alpha) * ptsA + alpha * ptsB
    points_morphed = clamp_points(points_morphed, SIZE, SIZE)

    # Triangulation de Delaunay
    rect = (0, 0, SIZE, SIZE)
    subdiv = cv2.Subdiv2D(rect)

    for p in points_morphed:
        x, y = float(p[0]), float(p[1])
        if 0 <= x < SIZE and 0 <= y < SIZE:
            subdiv.insert((x, y))

    triangle_list = subdiv.getTriangleList()

    tri_indices = []
    for t in triangle_list:
        tri_pts = [(t[0], t[1]), (t[2], t[3]), (t[4], t[5])]
        inds = []
        valid = True
        for p in tri_pts:
            idx = find_point_index(points_morphed, p, tol=5.0)
            if idx is None:
                valid = False
                break
            inds.append(idx)
        if valid and len(set(inds)) == 3:
            tri_indices.append(tuple(inds))

    tri_indices = list(set(tri_indices))

    # Morphing
    img_morphed = np.zeros_like(imgA_color, dtype=np.float32)

    for tri in tri_indices:
        i1, i2, i3 = tri
        tA = [ptsA[i1], ptsA[i2], ptsA[i3]]
        tB = [ptsB[i1], ptsB[i2], ptsB[i3]]
        tM = [points_morphed[i1], points_morphed[i2], points_morphed[i3]]

        if not (triangle_completely_inside(tA, SIZE, SIZE) and 
                triangle_completely_inside(tB, SIZE, SIZE) and 
                triangle_completely_inside(tM, SIZE, SIZE)):
            continue

        morph_triangle(imgA_color, imgB_color, img_morphed, tA, tB, tM, alpha)

    return np.clip(img_morphed, 0, 255).astype(np.uint8)

print("✓ Fonctions de morphing chargées")

In [ ]:
# ==================== FONCTIONS ARTISTIQUES ====================

def create_animated_gif(imgA, imgB, person_a, person_b, idx):
    """Crée un GIF animé montrant la transition entre deux visages"""
    
    # Générer une séquence de morphings
    steps = 15  # Nombre d'images dans l'animation
    alphas = np.linspace(0, 1, steps)
    
    frames = []
    
    for alpha in alphas:
        try:
            morphed = morph_faces(imgA, imgB, alpha=alpha)
            
            # Convertir en PIL Image
            morphed_rgb = cv2.cvtColor(morphed, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(morphed_rgb)
            
            # Agrandir pour mieux voir
            pil_img = pil_img.resize((256, 256), Image.LANCZOS)
            
            # Ajouter des informations avec PIL
            draw = ImageDraw.Draw(pil_img)
            
            # Texte avec progression
            text = f"{person_a[:12]} → {person_b[:12]}\nα = {alpha:.2f}"
            
            # Ajouter un rectangle semi-transparent en haut
            overlay = Image.new('RGBA', pil_img.size, (255, 255, 255, 0))
            overlay_draw = ImageDraw.Draw(overlay)
            overlay_draw.rectangle([(0, 0), (256, 40)], fill=(0, 0, 0, 180))
            pil_img = pil_img.convert('RGBA')
            pil_img = Image.alpha_composite(pil_img, overlay)
            pil_img = pil_img.convert('RGB')
            
            # Ajouter le texte
            draw = ImageDraw.Draw(pil_img)
            draw.text((10, 10), text, fill=(255, 255, 255))
            
            # Ajouter une barre de progression
            bar_width = int(256 * alpha)
            draw.rectangle([(0, 38), (bar_width, 40)], fill=(0, 255, 100))
            
            frames.append(np.array(pil_img))
            
        except Exception as e:
            print(f"⚠️  Erreur frame alpha={alpha:.2f}: {e}")
            continue
    
    # Ajouter les frames en sens inverse pour un effet de boucle
    frames = frames + frames[::-1]
    
    # Sauvegarder le GIF
    if len(frames) > 0:
        gif_path = GIFS_DIR / f"morph_{idx:04d}_{person_a[:8]}_{person_b[:8]}.gif"
        imageio.mimsave(gif_path, frames, fps=10, loop=0)
        return gif_path
    
    return None

def create_artistic_grid(morphed_images, metadata, filename="artistic_grid.png"):
    """Crée une grille artistique avec bordures et métadonnées"""
    
    rows, cols = GRID_SIZE
    n_images = min(len(morphed_images), rows * cols)
    
    if n_images == 0:
        return
    
    # Créer une figure avec fond personnalisé
    fig = plt.figure(figsize=(cols*4, rows*4))
    fig.patch.set_facecolor('#f0f0f0')
    
    gs = GridSpec(rows, cols, figure=fig, 
                 left=0.05, right=0.95, top=0.93, bottom=0.05,
                 wspace=0.3, hspace=0.4)
    
    # Titre principal
    fig.suptitle('🎭 GALERIE DE MORPHINGS FACIAUX 🎨', 
                fontsize=24, fontweight='bold', 
                color='#2c3e50', y=0.98)
    
    for i in range(n_images):
        ax = fig.add_subplot(gs[i // cols, i % cols])
        
        img = morphed_images[i]
        meta = metadata[i]
        
        # Afficher l'image
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
        # Titre avec style
        person_a = meta['person_a'][:12]
        person_b = meta['person_b'][:12]
        alpha = meta['alpha']
        
        title = f"{person_a}\n⬇\n{person_b}\nα={alpha:.2f}"
        ax.set_title(title, fontsize=10, fontweight='bold',
                    color='#34495e', pad=10)
        
        # Bordure artistique
        for spine in ax.spines.values():
            spine.set_edgecolor('#3498db')
            spine.set_linewidth(3)
        
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Remplir les espaces vides
    for i in range(n_images, rows * cols):
        ax = fig.add_subplot(gs[i // cols, i % cols])
        ax.axis('off')
    
    # Ajouter des statistiques en bas
    stats_text = f"Total: {n_images} morphings | Mode: {MODE} | Taille: {SIZE}x{SIZE}"
    fig.text(0.5, 0.01, stats_text, ha='center', fontsize=10, 
            color='#7f8c8d', style='italic')
    
    # Sauvegarder
    grid_path = GRIDS_DIR / filename
    plt.savefig(grid_path, dpi=200, bbox_inches='tight', 
               facecolor='#f0f0f0', edgecolor='none')
    plt.close()
    
    return grid_path

def add_fancy_frame(img, person_a, person_b, alpha, idx):
    """Ajoute un cadre artistique à une image"""
    
    # Convertir en PIL
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Agrandir l'image pour ajouter une marge
    margin = 20
    new_size = (SIZE + 2*margin, SIZE + 2*margin + 60)
    
    # Créer une nouvelle image avec fond blanc
    framed = Image.new('RGB', new_size, color='white')
    
    # Ajouter une ombre portée
    shadow = Image.new('RGBA', pil_img.size, color=(0, 0, 0, 0))
    shadow_enhanced = shadow.filter(ImageFilter.GaussianBlur(5))
    
    # Coller l'image
    framed.paste(pil_img, (margin, margin))
    
    # Ajouter des informations en bas
    draw = ImageDraw.Draw(framed)
    
    text_y = SIZE + 2*margin + 10
    text1 = f"#{idx:04d} | {person_a[:15]} ➜ {person_b[:15]}"
    text2 = f"Alpha: {alpha:.2f}"
    
    draw.text((margin, text_y), text1, fill=(50, 50, 50))
    draw.text((margin, text_y + 20), text2, fill=(100, 100, 100))
    
    # Dessiner une bordure
    draw.rectangle([(margin-2, margin-2), (SIZE+margin+2, SIZE+margin+2)], 
                  outline=(0, 150, 200), width=3)
    
    return framed

print("✓ Fonctions artistiques chargées")

In [ ]:
# ==================== GÉNÉRATION PRINCIPALE ====================

def generate_all_morphings():
    """Génère tous les morphings avec suivi en temps réel"""
    
    unique_labels = np.unique(labels)
    n_people = len(unique_labels)
    
    # Calculer le nombre de combinaisons
    if MODE == "all":
        total_combinations = n_people * (n_people - 1) // 2
        person_pairs = list(combinations(unique_labels, 2))
        
    elif MODE == "sample":
        total_combinations = min(NUM_SAMPLES, n_people * (n_people - 1) // 2)
        all_pairs = list(combinations(unique_labels, 2))
        person_pairs = [all_pairs[i] for i in np.random.choice(len(all_pairs), total_combinations, replace=False)]
        
    elif MODE == "per_person":
        total_combinations = n_people
        person_pairs = []
        for i, person_a in enumerate(unique_labels):
            person_b = unique_labels[(i + 1) % n_people]
            person_pairs.append((person_a, person_b))
    
    total_morphs = total_combinations * len(ALPHA_VALUES)
    
    # Affichage du plan de génération
    print(f"""
╔═══════════════════════════════════════════════════════════════════════╗
║                     PLAN DE GÉNÉRATION                                ║
╠═══════════════════════════════════════════════════════════════════════╣
║ 👥 Nombre de personnes    : {n_people:6d}                                    ║
║ 🔗 Paires à générer       : {total_combinations:6d}                                    ║
║ 🎨 Valeurs Alpha          : {len(ALPHA_VALUES):6d}                                    ║
║ 📊 Total morphings        : {total_morphs:6d}                                    ║
║                                                                       ║
║ 💾 Sauvegardes:                                                       ║
║    • Images individuelles : {'✓' if SAVE_INDIVIDUAL else '✗':5s}                                  ║
║    • GIFs animés          : {'✓' if CREATE_GIFS else '✗':5s}                                  ║
║    • Grille artistique    : {'✓' if CREATE_GRID else '✗':5s}                                  ║
╚═══════════════════════════════════════════════════════════════════════╝
""")
    
    # Confirmation pour de grandes générations
    if total_morphs > 100:
        print(f"⚠️  ATTENTION: {total_morphs} morphings vont être générés!")
        response = input("   Continuer? (o/N): ")
        if response.lower() not in ['o', 'oui', 'y', 'yes']:
            print("   ❌ Génération annulée.")
            return
    
    # Initialiser le tracker visuel
    print(f"\n🚀 Lancement de la génération...\n")
    time.sleep(1)  # Petite pause pour laisser l'utilisateur lire
    
    tracker = MorphingTracker(total_morphs)
    
    morphed_images = []
    metadata = []
    gif_paths = []
    
    try:
        # Parcourir toutes les paires
        for pair_idx, (person_a, person_b) in enumerate(person_pairs):
            
            # Récupérer les images
            imgs_a = images[labels == person_a]
            imgs_b = images[labels == person_b]
            
            # Sélectionner une image aléatoire pour chaque personne
            imgA = imgs_a[np.random.randint(len(imgs_a))]
            imgB = imgs_b[np.random.randint(len(imgs_b))]
            
            name_a = target_names[person_a]
            name_b = target_names[person_b]
            
            # Créer un GIF animé si demandé (une fois par paire)
            if CREATE_GIFS and pair_idx < 10:  # Limiter à 10 GIFs pour ne pas surcharger
                try:
                    gif_path = create_animated_gif(imgA, imgB, name_a, name_b, pair_idx)
                    if gif_path:
                        gif_paths.append(gif_path)
                except Exception as e:
                    print(f"\n⚠️  Erreur création GIF {name_a}+{name_b}: {e}")
            
            # Générer pour chaque alpha
            for alpha in ALPHA_VALUES:
                start_time = time.time()
                success = False
                img_morphed = None
                
                try:
                    img_morphed = morph_faces(imgA, imgB, alpha=alpha)
                    duration = time.time() - start_time
                    
                    # Sauvegarder individuellement si demandé
                    if SAVE_INDIVIDUAL:
                        filename = f"morph_{pair_idx:04d}_{name_a[:10]}_{name_b[:10]}_alpha{alpha:.2f}.png"
                        filepath = OUTPUT_DIR / filename
                        cv2.imwrite(str(filepath), img_morphed)
                    
                    # Ajouter à la liste pour la grille
                    morphed_images.append(img_morphed)
                    metadata.append({
                        'person_a': name_a,
                        'person_b': name_b,
                        'alpha': alpha,
                        'index': pair_idx
                    })
                    
                    success = True
                    
                except Exception as e:
                    duration = time.time() - start_time
                    print(f"\n⚠️  Échec: {name_a} + {name_b}: {e}")
                
                # Mettre à jour le tracker
                tracker.update(img_morphed, name_a, name_b, alpha, duration, success)
    
    finally:
        # Fermer le tracker
        tracker.close()
    
    # Résumé final
    elapsed_time = time.time() - tracker.start_time
    
    print(f"""

╔═══════════════════════════════════════════════════════════════════════╗
║                         GÉNÉRATION TERMINÉE ! ✅                      ║
╠═══════════════════════════════════════════════════════════════════════╣
║ ⏱️  Temps total          : {timedelta(seconds=int(elapsed_time)):>20s}             ║
║ ✅ Succès                : {tracker.success:6d} / {tracker.total:6d}                        ║
║ ❌ Échecs                : {tracker.failures:6d}                                    ║
║ ⚡ Vitesse moyenne       : {tracker.success/elapsed_time:6.2f} images/seconde              ║
║ 📈 Taux de réussite      : {100*tracker.success/max(1,tracker.total):6.1f}%                             ║
╚═══════════════════════════════════════════════════════════════════════╝
""")
    
    # Créer les visualisations finales
    if CREATE_GRID and len(morphed_images) > 0:
        print("\n📊 Création de la grille artistique...")
        grid_path = create_artistic_grid(morphed_images, metadata, 
                                        filename=f"grid_{MODE}_{len(morphed_images)}morphs.png")
        if grid_path:
            print(f"   ✓ Grille sauvegardée: {grid_path}")
    
    # Afficher les chemins des GIFs
    if len(gif_paths) > 0:
        print(f"\n🎬 {len(gif_paths)} GIFs animés créés:")
        for path in gif_paths[:5]:  # Afficher les 5 premiers
            print(f"   • {path.name}")
        if len(gif_paths) > 5:
            print(f"   ... et {len(gif_paths) - 5} autres")
    
    # Résumé des fichiers
    print(f"""
╔═══════════════════════════════════════════════════════════════════════╗
║                        FICHIERS GÉNÉRÉS                               ║
╠═══════════════════════════════════════════════════════════════════════╣
║ 📁 Dossier principal     : {str(OUTPUT_DIR):40s} ║
║ 🎬 GIFs animés           : {str(GIFS_DIR):40s} ║
║ 🖼️  Grilles artistiques   : {str(GRIDS_DIR):40s} ║
║                                                                       ║
║ 💡 ASTUCE: Consultez les GIFs pour voir les transitions animées !   ║
╚═══════════════════════════════════════════════════════════════════════╝
""")
    
    return morphed_images, metadata

print("✓ Fonction de génération principale chargée")

In [ ]:
# ==================== LANCEMENT ====================

def main():
    """Fonction principale avec menu interactif"""
    
    global MODE, NUM_SAMPLES, ALPHA_VALUES, CREATE_GIFS, CREATE_GRID
    
    print(f"""
╔═══════════════════════════════════════════════════════════════════════╗
║                      CONFIGURATION INTERACTIVE                        ║
╚═══════════════════════════════════════════════════════════════════════╝

Configuration actuelle:
  🎯 Mode             : {MODE}
  📊 Échantillons     : {NUM_SAMPLES}
  🎨 Alphas           : {ALPHA_VALUES}
  🎬 GIFs             : {'Oui' if CREATE_GIFS else 'Non'}
  🖼️  Grille           : {'Oui' if CREATE_GRID else 'Non'}
""")
    
    response = input("Modifier les paramètres? (o/N): ")
    
    if response.lower() in ['o', 'oui', 'y', 'yes']:
        print("""
╭──────────────────────────────────────────╮
│      MODES DE GÉNÉRATION                 │
├──────────────────────────────────────────┤
│ 1. sample      - Échantillon aléatoire   │
│ 2. per_person  - Un par personne         │
│ 3. all         - TOUTES les paires (!)   │
╰──────────────────────────────────────────╯
""")
        
        mode_choice = input("Choix (1/2/3) [défaut: 1]: ").strip() or "1"
        
        if mode_choice == "1":
            MODE = "sample"
            num_input = input(f"Nombre d'échantillons (1-1000) [défaut: {NUM_SAMPLES}]: ").strip()
            if num_input:
                NUM_SAMPLES = min(1000, max(1, int(num_input)))
        elif mode_choice == "2":
            MODE = "per_person"
        elif mode_choice == "3":
            MODE = "all"
            print("\n⚠️  ATTENTION: Mode 'all' peut générer des milliers d'images!")
        
        # Configuration des alphas
        print("\n🎨 Valeurs Alpha (séparées par des virgules)")
        print("   Exemples: 0.5  OU  0.3,0.5,0.7")
        alpha_input = input(f"Alphas [défaut: {','.join(map(str, ALPHA_VALUES))}]: ").strip()
        if alpha_input:
            ALPHA_VALUES = [float(x.strip()) for x in alpha_input.split(',')]
        
        # Options de sortie
        print("\n💾 Options de sortie")
        gifs_input = input(f"Créer des GIFs animés? (o/N) [défaut: {'o' if CREATE_GIFS else 'N'}]: ").strip()
        CREATE_GIFS = gifs_input.lower() in ['o', 'oui', 'y', 'yes']
        
        grid_input = input(f"Créer une grille? (o/N) [défaut: {'o' if CREATE_GRID else 'N'}]: ").strip()
        CREATE_GRID = grid_input.lower() in ['o', 'oui', 'y', 'yes']
    
    print("\n" + "="*71)
    print("🎬 DÉMARRAGE DE LA GÉNÉRATION")
    print("="*71 + "\n")
    
    # Lancer la génération
    morphed_images, metadata = generate_all_morphings()
    
    print("""
╔═══════════════════════════════════════════════════════════════════════╗
║                                                                       ║
║                    ✨ PROCESSUS TERMINÉ ! ✨                         ║
║                                                                       ║
║          Merci d'avoir utilisé Morphing Studio Amélioré !            ║
║                                                                       ║
╚═══════════════════════════════════════════════════════════════════════╝
""")

# Lancer le programme
if __name__ == "__main__":
    main()